In [ ]:
%load_ext autoreload
%autoreload 2

from DLC_for_WBFM.utils.projects.utils_project import load_config, safe_cd
from pathlib import Path
import zarr
import os
import pickle
import napari
import numpy as np

In [ ]:
# I think there are several sources that have the original indices, but let me check

# Option 1: The final traces objects

# Option 2: The final matches object

In [ ]:
project_fname = r'Y:\shared_projects\wbfm\dlc_stacks\Charlie-worm3-long\project_config.yaml'
project_cfg = load_config(project_fname)
traces_fname = project_cfg['subfolder_configs']['traces']
segment_fname = project_cfg['subfolder_configs']['segmentation']
project_dir = Path(project_fname).parent
with safe_cd(project_dir):
    trace_cfg = load_config(traces_fname)
    segment_cfg = load_config(segment_fname)

In [ ]:
# Get matches and segmentation
with safe_cd(project_dir):
#     fname = trace_cfg['all_matches']
#     with open(fname, 'rb') as f:
#         all_matches = pickle.load(f)
        
    fname = segment_cfg['output']['masks']
    masks = zarr.open(fname)
    
    segmentation_fname = segment_cfg['output']['metadata']
    with open(segmentation_fname, 'rb') as f:
        segmentation_metadata = pickle.load(f)

In [ ]:
viewer = napari.view_labels(masks)

In [ ]:
new_masks = zarr.zeros_like(masks)

In [ ]:
# Rename the first two volumes to be the global indices
# i.e. column 0 of the all_matches dict


# Build lookup table
# Note: if not all neurons are in the matches, then they are set to 0
def reindex_vol(i):
    # for i_volume, match in tqdm(all_matches.items()):
    match = all_matches[i]

    final_ind = match[:, 0].astype(int) + 1
    local_ind = match[:, 1].astype(int) + 1
    lut = np.zeros(1000, dtype=int)  # TODO: Should be more than the maximum local index
    # TODO: are the matches always the same length?
    lut[local_ind] = final_ind  # Raw indices of the lut should match the local index

    # Use lookup table
    new_masks[i, ...] = lut[masks[i, ...]]

In [ ]:
[reindex_vol(i) for i in range(10)]

In [ ]:
viewer = napari.view_labels(new_masks)

## Look at the DLC notations

In [ ]:
fname = r'Y:\shared_projects\wbfm\dlc_stacks\Charlie-immobilized-long\3-tracking\C-c10-C-2021-05-28\videos\center10DLC_resnet50_C-c10May28shuffle1_50000.h5'
import pandas as pd

df = pd.read_hdf(fname)

In [ ]:
# df.columns

# Look at the original segmentation metadata

In [ ]:
# all_matches[0]


# Look at reindexed masks

In [ ]:
# Get matches and segmentation
with safe_cd(project_dir):
    fname = os.path.join('4-traces', 'reindexed_masks.zarr')
    new_masks = zarr.open(fname)

In [ ]:
# viewer = napari.view_labels(new_masks)

# Part 2: reindexing just the tracklet training data

In [2]:
%load_ext autoreload
%autoreload 2

from DLC_for_WBFM.utils.projects.utils_project import load_config, safe_cd
from pathlib import Path
import zarr
import os
import pickle
import napari
import pandas as pd
import numpy as np
from DLC_for_WBFM.utils.visualization.utils_segmentation import all_matches_to_lookup_tables
from tqdm.auto import tqdm
from DLC_for_WBFM.utils.feature_detection.visualize_using_dlc import build_subset_df

In [3]:
project_fname = r'Y:\shared_projects\wbfm\dlc_stacks\Charlie-immobilized-long\project_config.yaml'
project_cfg = load_config(project_fname)
traces_fname = project_cfg['subfolder_configs']['traces']
segment_fname = project_cfg['subfolder_configs']['segmentation']
track_fname = project_cfg['subfolder_configs']['tracking']
project_dir = Path(project_fname).parent
with safe_cd(project_dir):
    trace_cfg = load_config(traces_fname)
    segment_cfg = load_config(segment_fname)
    track_cfg = load_config(track_fname)

In [4]:
# Get the frames chosen as training data
which_frames = track_cfg['training_data_3d']['which_frames']

# Get ALL matches to the segmentation, then subset
# Also get segmentation metadata
with safe_cd(project_dir):
    fname = os.path.join('2-training_data', 'raw', 'clust_df_dat.pickle')
    df = pd.read_pickle(fname)
    
    _get_frames_for_dlc_training(DEBUG, config, df)
    
    # Build a sub-df with only the relevant neurons; all slices
    subset_opt = {'which_z': None,
                  'max_z_dist': None,
                  'verbose': 1}
    subset_df = build_subset_df(df, which_frames, **subset_opt)
    
    fname = segment_cfg['output']['metadata']
    with open(fname, 'rb') as f:
        segmentation_metadata = pickle.load(f)
    
    fname = segment_cfg['output']['masks']
    masks = zarr.open(fname)

104 tracklets overlap in time
Keeping 104/327 tracklets


In [16]:
# Convert dataframe to matches per frame
all_matches = {}
for i, i_frame in tqdm(enumerate(which_frames)):
    matches = []
    for _, neuron_df in subset_df.iterrows():
        i_tracklet = neuron_df['all_ind_local'][i].astype(int)
        seg_ind = segmentation_metadata[i_frame].index[i_tracklet].astype(int)
        global_ind = neuron_df['clust_ind'] + 1
        matches.append([global_ind, seg_ind])
    all_matches[i_frame] = np.array(matches)

0it [00:00, ?it/s]

In [28]:
all_lut = all_matches_to_lookup_tables(all_matches)

new_sz = list(masks.shape)
new_sz[0] = len(which_frames)
# new_masks = zarr.zeros(new_sz, chunks=masks.chunks, dtype=int)
out_fname = os.path.join(project_dir, '2-training_data', 'reindexed_segmentation.zarr')
new_masks = zarr.open_like(masks, path=out_fname, shape=new_sz)

# Reindex
for i, (i_volume, lut) in tqdm(enumerate(all_lut.items())):
    new_masks[i, ...] = lut[masks[i_volume, ...]]


0it [00:00, ?it/s]

In [18]:
%gui

viewer = napari.view_labels(new_masks)


In [21]:

out_fname = os.path.join('2-training_data', 'reindexed_segmentation.zarr')
with safe_cd(project_dir):
    zarr.save(out_fname, new_masks)

In [32]:

out_fname = os.path.join('2-training_data', 'tmp.zarr')
with safe_cd(project_dir):
    z = zarr.open(out_fname)

In [33]:
z

<zarr.core.Array (31, 33, 133, 333) uint16>